In [3]:
from google.colab import drive
drive.mount('/content/drive')

!pip uninstall lightgbm
!pip install lightgbm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score
import warnings
warnings.filterwarnings(action='ignore')
from xgboost import XGBRegressor
!pip install catboost
import catboost

import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PowerTransformer,QuantileTransformer ,LabelEncoder, OneHotEncoder, MinMaxScaler, OrdinalEncoder, RobustScaler
import seaborn as sns
! pip install prince
import prince

! pip install optuna
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Uninstalling lightgbm-3.1.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/lightgbm-3.1.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/lightgbm/*
Proceed (y/n)? y
  Successfully uninstalled lightgbm-3.1.1
  Using cached https://files.pythonhosted.org/packages/70/cd/2b7783e8c250f8191b72e9a0010e0429a799d3305c27764d7bf113dfd078/lightgbm-3.1.1-py2.py3-none-manylinux1_x86_64.whl


In [4]:
path = '/content/drive/Shareddrives/dacon/Playground/'
train = pd.read_csv(path + 'train.csv', index_col=['id'])
test = pd.read_csv(path + 'test.csv', index_col=['id'])
submission = pd.read_csv(path + 'sample_submission.csv', index_col=['id'])

In [5]:
path2 = '/content/drive/Shareddrives/dacon/Playground/optimize/'
model3_train = pd.read_csv(path2 + 'trains_optuna_3model.csv', index_col=['Unnamed: 0'])
model3_predict = pd.read_csv(path2 + 'predictions_optuna_3model.csv', index_col=['Unnamed: 0'])

In [6]:
def create_stratified_folds_for_regression(data_df, n_splits=5):
    """
    @param data_df: training data to split in Stratified K Folds for a continous target value
    @param n_splits: number of splits
    @return: the training data with a column with kfold id
    """
    data_df['kfold'] = -1
    # randomize the data
    data_df = data_df.sample(frac=1, random_state=42).reset_index(drop=True)
    # calculate the optimal number of bins based on log2(data_df.shape[0])
    num_bins = np.int(np.floor(1 + np.log2(len(data_df))))
    # bins value will be the equivalent of class value of target feature used by StratifiedKFold to 
    # distribute evenly the classed over each fold
    data_df.loc[:, "bins"] = pd.cut(pd.to_numeric(data_df['target'], downcast="signed"), bins=num_bins, labels=False)
    kf = StratifiedKFold(n_splits=n_splits, random_state=42)
    
    # set the fold id as a new column in the train data
    for f, (t_, v_) in enumerate(kf.split(X=data_df, y=data_df.bins.values)):
        data_df.loc[v_, 'kfold'] = f
    
    # drop the bins column (no longer needed)
    data_df = data_df.drop("bins", axis=1)
    
    return data_df

In [7]:
def cv_evaluation_regression(df, model):
    n_fold = 5
    oof = np.zeros((len(df),))
    for fold in tqdm(range(n_fold)):
        val_ind = df[df.kfold == fold].index
        train = df[df.kfold != fold].reset_index(drop=True)
        valid = df[df.kfold == fold].reset_index(drop=True)

        features = [feature for feature in df.columns if feature not in ['id', 'target', 'kfold']]

        X_train = train[features]
        y_train = train['target']
        X_valid = valid[features]
        y_valid = valid['target']

        model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 500, verbose=False)
        preds = model.predict(valid[features])
        mse = mean_squared_error(y_valid, preds)
        
        oof[val_ind] = preds
    
    mse_oof = mean_squared_error(oof, df['target'])
    
    print(f'LGBM Overall RMSE:{np.sqrt(mse_oof)}')
    return np.sqrt(mse_oof)

In [8]:
train_data = train.iloc[:,:-1]
train_target = train.iloc[:, -1]
test_data = test.copy()

In [9]:
def dropper(dataset):
  df = dataset.copy()
  df['cat3'] = df['cat3'].apply(lambda x: x if x in ['A', 'C'] else 'Z')
  df['cat5'] = df['cat5'].apply(lambda x: x if x == 'C' else 'Z')
  df['cat6'] = df['cat6'].apply(lambda x: x if x in ['A', 'B', 'C'] else 'Z')
  df['cat7'] = df['cat7'].apply(lambda x: 'GE' if x in ['G', 'E'] else x)
  df['cat7'] = df['cat7'].apply(lambda x: 'BD' if x in ['B', 'D'] else x)
  df['cat8'] = df['cat8'].apply(lambda x: 'AD' if x in ['A', 'D'] else x)
  df['cat9'] = df['cat9'].apply(lambda x: 'BOK' if x in ['O', 'B', 'K'] else x)
  df['cat9'] = df['cat9'].apply(lambda x: 'NHM' if x in ['N', 'H', 'M'] else x)
  return df

In [10]:
train_temp = train_data.copy()
test_temp = test_data.copy()

cat_features = [feature for feature in train_temp.columns if 'cat' in feature]
cont_features = [feature for feature in train_temp.columns if 'cont' in feature]
scaler  = QuantileTransformer(random_state=42, n_quantiles=1500)

train_data[cont_features] = scaler.fit_transform(train_data[cont_features])
test_data[cont_features] = scaler.transform(test_data[cont_features])

train_data[cat_features] = dropper(train_temp[cat_features])
test_data[cat_features] = dropper(test_data[cat_features])

train_data = pd.concat([train_data, train_target], axis = 1)

In [11]:
train_data

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,,,,,
1,A,B,A,A,B,Z,A,GE,C,I,0.940916,0.743303,0.046189,0.262185,0.346990,0.900497,0.531811,0.895649,0.932840,0.844205,0.861846,0.678347,0.944570,0.721503,6.994023
2,B,A,A,A,B,Z,A,GE,AD,F,0.310632,0.015017,0.343941,0.973954,0.376355,0.495759,0.274397,0.253887,0.782310,0.590147,0.669928,0.800107,0.220230,0.922358,8.071256
3,A,A,A,C,B,Z,A,BD,C,NHM,0.830057,0.895601,0.493890,0.936372,0.409145,0.940410,0.362533,0.944315,0.888857,0.926020,0.142713,0.655557,0.930102,0.977543,5.760456
4,A,A,A,C,B,Z,A,GE,G,BOK,0.807019,0.937972,0.082587,0.770248,0.891234,0.962538,0.726620,0.544336,0.891863,0.816634,0.480207,0.664320,0.844488,0.582012,7.806457
6,A,B,A,A,B,Z,A,GE,C,F,0.470524,0.625662,0.598749,0.447376,0.188841,0.361440,0.837483,0.415459,0.546018,0.473193,0.861935,0.618260,0.317074,0.276864,6.868974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,A,B,A,C,B,Z,A,GE,E,L,0.059270,0.779462,0.093144,0.586359,0.889268,0.178653,0.064763,0.508377,0.223419,0.203659,0.102613,0.374541,0.006685,0.871086,8.343538
499996,A,B,A,C,B,Z,A,GE,E,L,0.019205,0.126351,0.797717,0.158230,0.900239,0.026820,0.062324,0.688836,0.114638,0.158871,0.173308,0.264305,0.132952,0.103993,7.851861
499997,A,B,A,C,B,Z,A,GE,C,NHM,0.203554,0.549040,0.514314,0.557816,0.413842,0.519039,0.422331,0.599026,0.046766,0.520089,0.102704,0.249794,0.017893,0.372121,7.600558


In [12]:
from sklearn import preprocessing

In [13]:
for cat in cat_features:
  le = preprocessing.LabelEncoder()
  le.fit(train_data[cat])
  train_data[cat] = le.transform(train_data[cat])
  test_data[cat] = le.transform(test_data[cat])

In [14]:
# Outlier 제거
trainset = train_data.copy()
trainset.drop(['target'], axis = 1, inplace  = True)
from sklearn.ensemble import IsolationForest
iso_model = IsolationForest(max_samples='auto', random_state=42, n_jobs = -1, contamination = 0.05, n_estimators=500)
iso_model.fit(trainset)

score = iso_model.decision_function(trainset)
anomaly = iso_model.predict(trainset)
train_data['scores'] = score
train_data['anomaly'] = anomaly

In [15]:
train_data

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,scores,anomaly
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,1,1,0,4,2,7,0.940916,0.743303,0.046189,0.262185,0.346990,0.900497,0.531811,0.895649,0.932840,0.844205,0.861846,0.678347,0.944570,0.721503,6.994023,0.065772,1
2,1,0,0,0,1,1,0,4,0,5,0.310632,0.015017,0.343941,0.973954,0.376355,0.495759,0.274397,0.253887,0.782310,0.590147,0.669928,0.800107,0.220230,0.922358,8.071256,-0.000638,-1
3,0,0,0,1,1,1,0,1,2,10,0.830057,0.895601,0.493890,0.936372,0.409145,0.940410,0.362533,0.944315,0.888857,0.926020,0.142713,0.655557,0.930102,0.977543,5.760456,0.007848,1
4,0,0,0,1,1,1,0,4,5,1,0.807019,0.937972,0.082587,0.770248,0.891234,0.962538,0.726620,0.544336,0.891863,0.816634,0.480207,0.664320,0.844488,0.582012,7.806457,0.056917,1
6,0,1,0,0,1,1,0,4,2,5,0.470524,0.625662,0.598749,0.447376,0.188841,0.361440,0.837483,0.415459,0.546018,0.473193,0.861935,0.618260,0.317074,0.276864,6.868974,0.105082,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,0,1,0,1,1,1,0,4,3,9,0.059270,0.779462,0.093144,0.586359,0.889268,0.178653,0.064763,0.508377,0.223419,0.203659,0.102613,0.374541,0.006685,0.871086,8.343538,0.065172,1
499996,0,1,0,1,1,1,0,4,3,9,0.019205,0.126351,0.797717,0.158230,0.900239,0.026820,0.062324,0.688836,0.114638,0.158871,0.173308,0.264305,0.132952,0.103993,7.851861,0.063895,1
499997,0,1,0,1,1,1,0,4,2,10,0.203554,0.549040,0.514314,0.557816,0.413842,0.519039,0.422331,0.599026,0.046766,0.520089,0.102704,0.249794,0.017893,0.372121,7.600558,0.090936,1


In [16]:
train_df = create_stratified_folds_for_regression(train_data)

In [17]:
train_df = train_df.drop(['anomaly'], axis = 1)

In [18]:
cat_model = lgb.LGBMRegressor(categorical_feature = [x for x in range(len(cat_features))], random_state=42)

In [19]:
train_df2 = train_df.drop(['scores'], axis =1)

In [20]:
cat_score = cv_evaluation_regression(train_df, cat_model)


LGBM Overall RMSE:0.8462878509592268


In [21]:
cat_score = cv_evaluation_regression(train_df2, cat_model)


LGBM Overall RMSE:0.8464686750067125


In [22]:
def objective_LGBM(trial: Trial, X):

  params = {} #initialize parameters
  params['random_state'] = 42  
  params['metric'] = 'rmse'
  params['n_jobs'] = -1
  params['max_bin'] = trial.suggest_int('max_bin', 256, 840)
  params['categorical_feature'] = [x for x in range(len(cat_features))]
  params['use_best_model'] = True
  params['reg_lambda'] = trial.suggest_uniform('reg_lambda', 0.001, 30)
  params['reg_alpha'] = trial.suggest_uniform('reg_alpha', 0.001, 30)
  params['colsample_bytree'] = trial.suggest_uniform('colsample_bytree', 0.1, 1)
  params['subsample'] = trial.suggest_uniform('subsample', 0.1, 1)
  params['learning_rate'] = trial.suggest_categorical('learning_rate', [0.003,0.009,0.01,0.012,0.014,0.016,0.018, 0.02])
  params['n_estimators'] = trial.suggest_categorical('n_estimators', [500,750,1000,1500,2000,3000,4000,5000, 10000, 20000])
  params['max_depth'] = trial.suggest_int('max_depth', 5, 25)
  params['num_leaves'] = trial.suggest_int('num_leaves', 5, 200)
  params['min_child_weight'] = trial.suggest_uniform('min_child_weight', 0.0001, 100)
  params['cat_smooth'] = trial.suggest_int('cat_smooth', 0, 50)
  params['cat_l2'] = trial.suggest_uniform('cat_l2', 0.001, 30)
  params['min_child_samples'] = trial.suggest_int('min_child_samples', 1, 300)

  model = lgb.LGBMRegressor(**params)
  score = cv_evaluation_regression(X, model)

  return score

In [24]:
study = optuna.create_study(direction = 'minimize')

[I 2021-02-20 12:29:15,228] A new study created in memory with name: no-name-275da362-ae0d-4b09-b748-b76ba57c143f


In [25]:
X_train = train.drop(['target'], axis = 1)
y_train = train.target
X_test = test.copy()

cat_cols = [feature for feature in train.columns if 'cat' in feature]

def label_encoder(df):
    for feature in cat_cols:
        le = LabelEncoder()
        le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df
X_train = label_encoder(X_train)
X_test = label_encoder(X_test)

In [29]:
train_df3 = pd.concat([X_train, y_train], axis =1)
train_df3 = create_stratified_folds_for_regression(train_df3)
train_df3

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,0,0,0,1,1,0,4,4,5,0.517321,0.649070,0.495282,0.407344,0.281229,0.377936,0.695889,0.225072,0.279877,0.397533,0.346235,0.318158,0.322053,0.731761,5.733432,0
1,0,0,0,0,1,3,0,4,4,13,0.311191,0.049718,0.518400,0.157790,0.298866,0.398920,0.541013,0.261724,0.300421,0.210114,0.356237,0.298105,0.294468,0.757850,6.990234,0
2,0,0,1,0,1,3,0,4,6,13,0.869850,0.678942,0.734321,0.209937,0.277330,0.835372,0.328593,0.777681,0.266530,0.654293,0.464221,0.698295,0.831430,0.798729,8.212900,0
3,0,0,0,2,1,1,0,4,4,5,0.347573,0.131831,0.315201,0.745363,0.419922,0.745390,0.384305,0.251359,0.249770,0.334739,0.386494,0.358721,0.305164,0.756006,8.466549,0
4,0,0,0,2,1,1,0,4,3,8,0.424749,0.696019,0.949583,0.424285,0.411343,0.227085,0.367200,0.271663,0.302262,0.316547,0.528495,0.290432,0.331302,0.308376,7.449541,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0,0,0,2,1,1,0,4,4,0,0.218591,0.134988,0.662657,0.245763,0.274016,0.609485,0.242897,0.244675,0.337680,0.326358,0.340959,0.315536,0.361444,0.355215,6.611676,4
299996,0,1,0,0,1,3,0,4,4,8,0.317958,0.666808,0.645005,0.357515,0.277544,0.226717,0.497106,0.308617,0.476655,0.242522,0.698014,0.757678,0.648951,0.406966,7.253953,4
299997,0,0,0,2,1,1,0,4,0,0,0.477097,0.673851,0.358404,0.751019,0.280375,0.382030,0.912317,0.587112,0.448978,0.448227,0.748775,0.741487,0.209404,0.712498,6.979163,4
299998,0,1,0,2,1,1,0,4,2,11,0.277819,0.354847,0.801441,0.509487,0.374181,0.252294,0.396459,0.491125,0.321046,0.319039,0.373475,0.355155,0.660828,0.210733,6.423450,4


In [30]:
lgbm_params = {'max_depth': 16, 
                'subsample': 0.8032697250789377, 
                'colsample_bytree': 0.21067140508531404, 
                'learning_rate': 0.009867383057779643,
                'reg_lambda': 10.987474846877767, 
                'reg_alpha': 17.335285595031994, 
                'min_child_samples': 31, 
                'num_leaves': 66, 
                'max_bin': 522, 
                'cat_smooth': 81, 
                'cat_l2': 0.029690334194270022, 
                'metric': 'rmse', 
                'n_jobs': -1, 
                'n_estimators': 20000,
                'random_state':42}

k_model = lgb.LGBMRegressor(**lgbm_params)

In [31]:
cat_score = cv_evaluation_regression(train_df3, k_model)


LGBM Overall RMSE:0.8421059036171497


In [35]:
study.optimize(lambda trial : objective_LGBM(trial, train_df), n_trials=70)

[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model


[I 2021-02-20 17:50:05,727] Trial 64 finished with value: 0.8425282036264969 and parameters: {'max_bin': 547, 'reg_lambda': 12.458319308377705, 'reg_alpha': 23.488215231862412, 'colsample_bytree': 0.13427601898316455, 'subsample': 0.41454989829166744, 'learning_rate': 0.012, 'n_estimators': 4000, 'max_depth': 24, 'num_leaves': 72, 'min_child_weight': 0.2992945407435884, 'cat_smooth': 33, 'cat_l2': 18.39583697968466, 'min_child_samples': 233}. Best is trial 24 with value: 0.842088761117545.



LGBM Overall RMSE:0.8425282036264969


[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model


[I 2021-02-20 17:55:47,520] Trial 65 finished with value: 0.842246812760171 and parameters: {'max_bin': 497, 'reg_lambda': 18.03543737168221, 'reg_alpha': 29.548911321991888, 'colsample_bytree': 0.18577814689106828, 'subsample': 0.2342443442257755, 'learning_rate': 0.012, 'n_estimators': 4000, 'max_depth': 24, 'num_leaves': 86, 'min_child_weight': 5.113640596787697, 'cat_smooth': 28, 'cat_l2': 20.367837945278833, 'min_child_samples': 246}. Best is trial 24 with value: 0.842088761117545.


...

[I 2021-02-21 00:41:44,050] Trial 132 finished with value: 0.8421225879924259 and parameters: {'max_bin': 587, 'reg_lambda': 26.731912265070388, 'reg_alpha': 22.136135621205547, 'colsample_bytree': 0.20991485868153212, 'subsample': 0.44971640450866546, 'learning_rate': 0.012, 'n_estimators': 10000, 'max_depth': 24, 'num_leaves': 43, 'min_child_weight': 76.3058488744744, 'cat_smooth': 39, 'cat_l2': 23.473372935191318, 'min_child_samples': 218}. Best is trial 24 with value: 0.842088761117545.



LGBM Overall RMSE:0.8421225879924259


[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model


[I 2021-02-21 00:47:26,604] Trial 133 finished with value: 0.8421054609285916 and parameters: {'max_bin': 595, 'reg_lambda': 28.084413438717984, 'reg_alpha': 22.446623616324388, 'colsample_bytree': 0.19346612257989487, 'subsample': 0.6363326749092639, 'learning_rate': 0.012, 'n_estimators': 10000, 'max_depth': 24, 'num_leaves': 55, 'min_child_weight': 79.62441977641697, 'cat_smooth': 29, 'cat_l2': 14.570788759746796, 'min_child_samples': 216}. Best is trial 24 with value: 0.842088761117545.



LGBM Overall RMSE:0.8421054609285916


In [33]:
optuna.visualization.plot_optimization_history(study)

In [34]:
study.best_params

{'cat_l2': 24.16256018684644,
 'cat_smooth': 19,
 'colsample_bytree': 0.18021264437889178,
 'learning_rate': 0.012,
 'max_bin': 589,
 'max_depth': 22,
 'min_child_samples': 255,
 'min_child_weight': 17.408465847532234,
 'n_estimators': 4000,
 'num_leaves': 65,
 'reg_alpha': 22.912590049548058,
 'reg_lambda': 18.645825925028017,
 'subsample': 0.990962372303398}

In [36]:
study.best_params

{'cat_l2': 24.16256018684644,
 'cat_smooth': 19,
 'colsample_bytree': 0.18021264437889178,
 'learning_rate': 0.012,
 'max_bin': 589,
 'max_depth': 22,
 'min_child_samples': 255,
 'min_child_weight': 17.408465847532234,
 'n_estimators': 4000,
 'num_leaves': 65,
 'reg_alpha': 22.912590049548058,
 'reg_lambda': 18.645825925028017,
 'subsample': 0.990962372303398}

In [ ]:
study.optimize(lambda trial : objective_LGBM(trial, train_df), n_trials=70)

[LightGBM] [Warning] min_data_in_leaf is set with min_data=571, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=571
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=571, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=571
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=571, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=571
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=571, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=571
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=571, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=571
[LightGBM] [Warning] Unknown parameter: use_best_model


[I 2021-02-19 23:53:20,479] Trial 70 finished with value: 0.842914710028268 and parameters: {'min_data': 571, 'reg_lambda': 6.007849226437598, 'reg_alpha': 5.990046492031584, 'colsample_bytree': 0.11798264177557036, 'subsample': 0.603257365648692, 'learning_rate': 0.01, 'n_estimators': 3000, 'max_depth': 24, 'num_leaves': 102, 'min_child_weight': 31.118213240516457, 'cat_smooth': 11}. Best is trial 31 with value: 0.8421649367455053.



LGBM Overall RMSE:0.842914710028268


[LightGBM] [Warning] min_data_in_leaf is set with min_data=480, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=480
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=480, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=480
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=480, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=480
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=480, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=480
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=480, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=480
[LightGBM] [Warning] Unknown parameter: use_best_model


[I 2021-02-19 23:58:52,580] Trial 71 finished with value: 0.8421855905649419 and parameters: {'min_data': 480, 'reg_lambda': 6.9067287510753825, 'reg_alpha': 10.113681696808461, 'colsample_bytree': 0.20285879340315452, 'subsample': 0.33737497430699304, 'learning_rate': 0.009, 'n_estimators': 3000, 'max_depth': 12, 'num_leaves': 69, 'min_child_weight': 75.41314971819489, 'cat_smooth': 2}. Best is trial 31 with value: 0.8421649367455053.


...

[I 2021-02-20 05:31:59,430] Trial 138 finished with value: 0.8424693506089246 and parameters: {'min_data': 532, 'reg_lambda': 0.8931747303667135, 'reg_alpha': 15.280522936469845, 'colsample_bytree': 0.2472341460016345, 'subsample': 0.6805474431257857, 'learning_rate': 0.009, 'n_estimators': 5000, 'max_depth': 10, 'num_leaves': 73, 'min_child_weight': 85.69626010695833, 'cat_smooth': 40}. Best is trial 133 with value: 0.8421477998074605.



LGBM Overall RMSE:0.8424693506089246


[LightGBM] [Warning] min_data_in_leaf is set with min_data=506, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=506
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=506, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=506
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=506, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=506
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=506, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=506
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=506, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=506
[LightGBM] [Warning] Unknown parameter: use_best_model


[I 2021-02-20 05:37:00,063] Trial 139 finished with value: 0.8429264980372552 and parameters: {'min_data': 506, 'reg_lambda': 6.274918221020855, 'reg_alpha': 13.19454195980702, 'colsample_bytree': 0.15463233394601297, 'subsample': 0.6442259906558052, 'learning_rate': 0.01, 'n_estimators': 5000, 'max_depth': 11, 'num_leaves': 65, 'min_child_weight': 81.42793803848646, 'cat_smooth': 48}. Best is trial 133 with value: 0.8421477998074605.



LGBM Overall RMSE:0.8429264980372552
